<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-Libraries" data-toc-modified-id="Import-Libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import Libraries</a></span></li><li><span><a href="#Import-scraped-data" data-toc-modified-id="Import-scraped-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Import scraped data</a></span></li><li><span><a href="#Things-To-Do" data-toc-modified-id="Things-To-Do-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Things To Do</a></span></li></ul></div>

# Import Libraries

In [2]:
import numpy as np 
import pandas as pd

import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib.pylab import rcParams

plt.style.use('ggplot')
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import itertools
import statsmodels.api as sm
import operator

from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from xgboost import XGBClassifier
import xgboost as xgb
from xgboost import plot_importance

from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix

import time

from bs4 import BeautifulSoup
import requests
import urllib.request
from urllib.request import Request, urlopen

# Import scraped data

In [3]:
df = pd.read_csv('AN_coffee_reviews.csv')

In [4]:
df.head()

,Unnamed: 0,Acidity,Acidity /Structure,Aftertaste,Agtron,Aroma,Body,Coffee Origin,Est. Price,Flavor,Review Date,Roast Level,Roaster Location,With Milk,bottom_line,rating,roaster,title
0,0,NaN,9.0,8.0,58/76,9,9,"Tarrazu, Costa Rica",$23.00/12 ounces,9,April 2020,Medium-Light,"Billings, Montana",NaN,Enjoying this Costa Rica evokes the pleasures ...,94,Revel Coffee,Costa Rica Luis Campos Anaerobic
1,1,NaN,9.0,8.0,60/74,9,9,"Valle del Cauca growing region, Colombia",$23.50/12 ounces,9,April 2020,Medium-Light,"Topeka, Kansas",NaN,A bombshell of a coffee with enough personalit...,94,PT's Coffee Roasting Co.,Colombia Granja La Esperanza Tres Dragones
2,2,NaN,8.0,8.0,50/70,9,8,"Nyeri growing region, south-central Kenya",NT $530/200 grams,9,April 2020,Medium,"Taipei, Taiwan",NaN,A developed medium roast emphasizes this Kenya...,92,Quartet Kaffe,Kenya Thageini
3,3,NaN,9.0,8.0,58/76,9,9,"Huila, Colombia",$16.50/12 ounces,9,March 2020,Medium-Light,"Jackson, Mississippi",NaN,An exceptional Colombia cup with sweet and tar...,94,BeanFruit Coffee Co.,Colombia Finca La Loma Microlot
4,4,NaN,NaN,7.0,56/67,7,8,Sumatra,$12.36/12 ounces,8,March 2020,Medium-Light,"Telluride, Colorado",8,"The sweet, fruit-on-the-edge of decay characte...",88,Telluride Coffee Roasters,Aged Sumatra Espresso


# Things To Do

In [5]:
# redo proposal
# set index
# fix columns (reorder,drop)
# fix data types 
# fix pricing (first set null values to 0, )
# check for missing data / nulls
# potentially change some column names
# visualizations

In [6]:
# df = pd.DataFrame(coffees1)
# df = df.drop(['With Milk'], axis=1)
# df = df[['title','roaster','Roaster Location','Coffee Origin','rating','Roast Level','Review Date','Est. Price',
#          'Agtron','Flavor','Body','Aroma','Aftertaste','Acidity /Structure', 'Acidity ','With Milk']]

In [7]:
#df[df['Est. Price'].isnull()]